This tutorial is available on the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-run-anywhere/pipeline-architecture/wallaroo-gpu-llm-summarization).

## LLM Summarization GPU Edge Deployment

This tutorial demonstrates how to use the Wallaroo combined with GPU processors to perform inferences with pre-trained computer vision ML models.  This demonstration assumes that:

* Wallaroo Version 2023.3 or above instance is installed.
* A nodepools with GPUs part of the Kubernetes cluster.  See [Create GPU Nodepools for Kubernetes Clusters](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-install-guides/wallaroo-install-configurations/wallaroo-gpu-nodepools/) for more detials.
* The model [`hf-summarization-bart-large-samsun.zip` (1.4 G)](https://storage.googleapis.com/wallaroo-public-data/llm-models/model-auto-conversion_hugging-face_complex-pipelines_hf-summarisation-bart-large-samsun.zip) has been downloaded to the `./models` folder.

### Tutorial Goals

For our example, we will perform the following:

* Create a workspace for our work.
* Upload the model.
* Create a pipeline and specify the gpus in the pipeline deployment.
* Perform a sample inference.


## Steps

### Import Libraries

The first step will be to import our libraries.

In [10]:
import json
import os

import wallaroo
from wallaroo.pipeline   import Pipeline
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [11]:
wl = wallaroo.Client()

### Configure PyArrow Schema

You can find more info on the available inputs under [TextSummarizationInputs](https://github.com/WallarooLabs/platform/blob/main/conductor/model-auto-conversion/flavors/hugging-face/src/io/pipeline_inputs/text_summarization_inputs.py#L14) or under the [official source code](https://github.com/huggingface/transformers/blob/v4.28.1/src/transformers/pipelines/text2text_generation.py#L241) from `🤗 Hugging Face`.

In [12]:
input_schema = pa.schema([
    pa.field('inputs', pa.string()),
    pa.field('return_text', pa.bool_()),
    pa.field('return_tensors', pa.bool_()),
    pa.field('clean_up_tokenization_spaces', pa.bool_()),
    # pa.field('generate_kwargs', pa.map_(pa.string(), pa.null())), # dictionaries are not currently supported by the engine
])

output_schema = pa.schema([
    pa.field('summary_text', pa.string()),
])

### Upload Model

We will now create or connect to our pipeline and upload the model.

The model's AI Accelerator is set during the model upload process.  For this model, that is set to `CUDA`.

In [14]:
model = wl.upload_model('hf-summarization', 
                        './models/hf-summarisation-bart-large-samsun.zip', 
                        framework=Framework.HUGGING_FACE_SUMMARIZATION, 
                        input_schema=input_schema, 
                        output_schema=output_schema,
                        accel=wallaroo.engine_config.Acceleration.CUDA)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a container runtime..
Model is attempting loading to a container runtime.....................................................successful

Ready


Name,hf-summarization-yns
Version,9297e44f-4c7a-4055-9a90-5284d3bf46a5
File Name,hf-summarisation-bart-large-samsun.zip
SHA,ee71d066a83708e7ca4a3c07caf33fdc528bb000039b6ca2ef77fa2428dc6268
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2024.1.0-main-4921
Architecture,x86
Acceleration,cuda
Updated At,2024-17-Apr 19:46:06


### Deploy Pipeline

With the model uploaded, we can add it is as a step in the pipeline, then deploy it.  

For GPU deployment, the pipeline deployment configuration allocated the cpus, ram, gpus, and other settings for the pipeline.  For gpus,  both the number of GPUs and the nodepool containing the gpus must be specified.

For Wallaroo Native Runtime models (`onnx`, `tensorflow`), the method is `wallaroo.deployment_config.gpus(int)` to allocate the number of gpus to the pipeline.  This applies to all Wallaroo Native Runtime models in the pipeline.

For Wallaroo Containerized models (`hugging-face`, etc), the method is `wallaroo.deployment_config.sidekick_gpus(int)` to allocate the number of gpus to the model.

The deployment label is set with the `wallaroo.deployment_config.deployment_label(string)` method.

For more information on allocating resources to a Wallaroo pipeline for deployment, see [Wallaroo SDK Essentials Guide: Pipeline Deployment Configuration](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-deployment-config/).

For this example, 1 gpu will be allocated to the pipeline from the nodepool with the deployment label `wallaroo.ai/accelerator: a100`.

Note that the accelerator setting for the deployment configuration is not specified; this is inherited from the model's `accel` parameter.

In [15]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(1).memory('1Gi') \
    .sidekick_gpus(model, 1) \
    .sidekick_cpus(model,4) \
    .sidekick_memory(model, '8Gi') \
    .deployment_label('wallaroo.ai/accelerator: a100') \
    .build()

In [23]:
deployment_config

{'engine': {'cpu': 1,
  'resources': {'limits': {'cpu': 1, 'memory': '1Gi'},
   'requests': {'cpu': 1, 'memory': '1Gi'}},
  'node_selector': 'wallaroo.ai/accelerator: a100',
  'arch': 'x86',
  'accel': 'cuda'},
 'enginelb': {},
 'engineAux': {'images': {'hf-summarization-yns-65': {'resources': {'limits': {'nvidia.com/gpu': 1,
      'cpu': 4,
      'memory': '8Gi'},
     'requests': {'nvidia.com/gpu': 1, 'cpu': 4, 'memory': '8Gi'}}}}},
 'node_selector': {}}

In [18]:
pipeline_name = "hf-summarization-pipeline"
pipeline = wl.build_pipeline(pipeline_name)
pipeline.add_model_step(model)

name,hf-summarization-pipeline
created,2024-04-17 19:47:45.965024+00:00
last_updated,2024-04-17 19:48:53.441612+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,"34289130-76ca-47c1-a672-9e6b027056d5, 8a9b269a-9b0a-4d68-a591-46f97392de0f, 8b79fb5a-2743-4e3a-a2e4-c8ac32884394"
steps,
published,False


In [20]:
pub = pipeline.publish(deployment_config)

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing............................. Published.


In [22]:
display(pub)

ID,1
Pipeline Name,hf-summarization-pipeline
Pipeline Version,6d453276-a4cf-4b01-90d7-78e9da1dd72a
Status,Published
Engine URL,ghcr.io/wallaroolabs/doc-samples/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini-cuda:v2024.1.0-main-4921
Pipeline URL,ghcr.io/wallaroolabs/doc-samples/pipelines/hf-summarization-pipeline:6d453276-a4cf-4b01-90d7-78e9da1dd72a
Helm Chart URL,oci://ghcr.io/wallaroolabs/doc-samples/charts/hf-summarization-pipeline
Helm Chart Reference,ghcr.io/wallaroolabs/doc-samples/charts@sha256:a9406689f7429c16758447780c860ee41c78dc674280754eb2b377da1a9efbf4
Helm Chart Version,0.0.1-6d453276-a4cf-4b01-90d7-78e9da1dd72a
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 1.0, 'memory': '512Mi'}, 'accel': 'cuda', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': {}}}"
User Images,[]


In [21]:
display(pipeline)

name,hf-summarization-pipeline
created,2024-04-17 19:47:45.965024+00:00
last_updated,2024-04-17 19:49:31.864182+00:00
deployed,(none)
arch,None
accel,None
tags,
versions,"6d453276-a4cf-4b01-90d7-78e9da1dd72a, c2a5693d-a580-45be-8898-5dd7ee36cb00, 34289130-76ca-47c1-a672-9e6b027056d5, 8a9b269a-9b0a-4d68-a591-46f97392de0f, 8b79fb5a-2743-4e3a-a2e4-c8ac32884394"
steps,
published,False


In [ ]:
pipeline.deploy(deployment_config=deployment_config)

### Run inference

We can now run a sample inference using the `wallaroo.pipeline.infer` method and display the results.

In [8]:
input_data = {
        "inputs": ["LinkedIn (/lɪŋktˈɪn/) is a business and employment-focused social media platform that works through websites and mobile apps. It launched on May 5, 2003. It is now owned by Microsoft. The platform is primarily used for professional networking and career development, and allows jobseekers to post their CVs and employers to post jobs. From 2015 most of the company's revenue came from selling access to information about its members to recruiters and sales professionals. Since December 2016, it has been a wholly owned subsidiary of Microsoft. As of March 2023, LinkedIn has more than 900 million registered members from over 200 countries and territories. LinkedIn allows members (both workers and employers) to create profiles and connect with each other in an online social network which may represent real-world professional relationships. Members can invite anyone (whether an existing member or not) to become a connection. LinkedIn can also be used to organize offline events, join groups, write articles, publish job postings, post photos and videos, and more"], # required
        "return_text": [True], # optional: using the defaults, similar to not passing this parameter
        "return_tensors": [False], # optional: using the defaults, similar to not passing this parameter
        "clean_up_tokenization_spaces": [False], # optional: using the defaults, similar to not passing this parameter
}
dataframe = pd.DataFrame(input_data)
dataframe

,inputs,return_text,return_tensors,clean_up_tokenization_spaces
0,LinkedIn (/lɪŋktˈɪn/) is a business and employ...,True,False,False


In [9]:
out = pipeline.infer(dataframe)
out

,time,in.clean_up_tokenization_spaces,in.inputs,in.return_tensors,in.return_text,out.summary_text,check_failures
0,2023-09-08 13:18:58.557,False,LinkedIn (/lɪŋktˈɪn/) is a business and employ...,False,True,LinkedIn is a business and employment-focused ...,0


In [11]:
out["out.summary_text"][0]

'LinkedIn is a business and employment-focused social media platform that works through websites and mobile apps. It launched on May 5, 2003. LinkedIn allows members (both workers and employers) to create profiles and connect with each other in an online social network which may represent real-world professional relationships.'

### Model Inferencing with Pipeline Deployment Endpoint

The other option is to use the pipeline's inference endpoint.

In [13]:
!curl -X POST {pipeline.url()} \
    -H "Content-Type: application/json; format=pandas-records" \
        -d @./data/test_summarization.json

[{"time":1694179270999,"in":{"clean_up_tokenization_spaces":[false],"inputs":["LinkedIn (/lɪŋktˈɪn/) is a business and employment-focused social media platform that works through websites and mobile apps. It launched on May 5, 2003. It is now owned by Microsoft. The platform is primarily used for professional networking and career development, and allows jobseekers to post their CVs and employers to post jobs. From 2015 most of the company's revenue came from selling access to information about its members to recruiters and sales professionals. Since December 2016, it has been a wholly owned subsidiary of Microsoft. As of March 2023, LinkedIn has more than 900 million registered members from over 200 countries and territories. LinkedIn allows members (both workers and employers) to create profiles and connect with each other in an online social network which may represent real-world professional relationships. Members can invite anyone (whether an existing member or not) to become a co

### Undeploy the Pipeline

With the demonstration complete, we can undeploy the pipeline and return the resources back to the Wallaroo instance.

In [ ]:
pipeline.undeploy()